In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio

In [10]:
import numpy as np
import scipy.io.wavfile
import time
import pyaudio
import IPython.display as ipd

# roll array x by k elements
# zero out the last k elements of x
def roll_zero(x, k):
  result = x[k:]
  result = np.append(x[k:], np.zeros(k))
  return result

#zero-padding the signal (0's at the end)
def zero_pad(x, k):
  return np.append(x, np.zeros(k))

# partition impulse response and precompute frequency response for each block
def precompute_frequency_responses(h, L, k, num_blocks):
  H = np.zeros((num_blocks, L+k)).astype('complex128')
  for j in range(num_blocks):
    H[j] += np.fft.fft(zero_pad(h[j*k: (j+1)*k], L))
  return H

In [23]:
# Frequency Domain Delay Line
# Do Overlap Add in Frequency Domain
def fdl(x, h):
  L = 2**8 #signal block size
  p = len(h)
  k = L # ir block size

  num_ir_blocks = int(p/k)
  num_sig_blocks = int(len(x) / L)

  #compute the freq_resp of the IR
  H = precompute_frequency_responses(h, L, k, num_ir_blocks)

  fdl = np.zeros(2*L*num_ir_blocks).astype('complex128')
  output = np.zeros(p+len(x)-1).astype('float64')
  out = np.zeros(2*L-1)

  for i in range(num_sig_blocks):
    input_buffer = x[i*L: (i+1)*L]
    spectrum = np.fft.fft(zero_pad(input_buffer, L))
    
    for j in range(num_ir_blocks):
      fdl[j*2*L: (j+1)*2*L] += H[j] * spectrum

    out += np.fft.ifft(fdl[:2*L]).real[:2*L-1]
    output[i*L:(i+1)*L] += out[:L]
    fdl = roll_zero(fdl, 2*L)
    out = roll_zero(out, L)

  for i in range(1, num_ir_blocks): #process remaining frequency blocks
    out += np.fft.ifft(fdl[:2*L]).real[:2*L-1]
    output[num_sig_blocks+i*L: num_sig_blocks+(i+1)*L] += out[:L]
    out = roll_zero(out, L)
    fdl = roll_zero(fdl, 2*L)
    
  x_zp = zero_pad(x, p-1)
  output = amount_verb  * output + x_zp
  return output

In [24]:
fs1, guitar = scipy.io.wavfile.read('/content/sample_data/blues_guitar.wav')
fs2, reverb = scipy.io.wavfile.read('/content/sample_data/Conic Long Echo Hall.wav')
reverb = (reverb.astype('float64') / np.max(reverb))[:,0] #one channel only
amount_verb = .9

sig4 = fdl(guitar, reverb)

<ipython-input-24-fa7dd40bfc1a>:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs1, guitar = scipy.io.wavfile.read('/content/sample_data/blues_guitar.wav')


In [ ]:
ipd.display(ipd.Audio(data=sig4, rate=fs1))